# ห้ามลบ

- ในหน้านี้พลเป็นคนทำ
- ควรแก้ ในการยิง API ผมได้ใส่ parametor link ให้คุณเอา link สับย่อยมาใส่

##สิ่งที่แก้ไข

1.ทำการลดการ import

2.ลบหรือยุบตัวแปรที่ไม่ได้ใช้ออกเพื่อให้เข้าใจง่าย

3.เปลี่ยนแปลง API  ให้สามารถส่งลิ้งก์มาด้วย

4.ได้ทำการเปลี่ยน nan เป็น "" หรือ 0 สำเร็จแล้ว

5.uri_API  เพิ่มเส้น APi ไว้ด้านบนใต้ import เพื่อให้ง่ายในการแก้ไข

In [11]:
import requests,pandas as pd
uri_API = 'http://26.158.60.250:8080/';
data_link_for_lazada  = {
    0:'อุปกรณ์-อิเล็กทรอนิกส์',
    1:'อุปกรณ์เสริม-อิเล็กทรอนิกส์',
    2:'ทีวีและเครื่องใช้ในบ้าน',
    3:'สุขภาพและความงาม',
    4:'ทารกและของเล่น',
    5:'ของชำและสัตว์เลี้ยง',
    6:'บ้านและไลฟ์สไตล์',
    7:'แฟชั่นและเครื่องประดับผู้หญิง',
    8:'แฟชั่นและเครื่องประดับผู้ชาย',
    9:'กีฬาและการเดินทาง',
    10:'ยานยนต์และรถจักรยานยนต์', 
    11:"ตั๋วและบัตรกำนัน"
}
header_2 = ['_95X4G href', 'Image', 'jBwCF src 2'
          , 'Data', 'Price',
       'Sold', 'Amount', 'Place']
header_Values = {
    '_95X4G href':"product",
    'Image':"image_product_1",
    'jBwCF src 2':"image_product_2",
    'IcOsH':"discount",
    'Data':"data_product", 
    'Price':"price_product",
    'Sold':"sold",
    'Amount':"count_review", 
    'Place':"place"
}
status_run_program= False

In [12]:
import requests,os,json,time,pandas as pd,pyautogui,shutil,pyperclip,keyboard as ky
from datetime import datetime
from fnmatch import fnmatch
from pynput import keyboard,mouse
from data_address import address as ad

bot_lazada = r'\Bot_lazada'
path_file = os.getcwd();
drag_data = os.path.abspath(os.path.join(path_file, os.pardir))
data_lazada = r'\Data_lazada';
data_lazada_xlsx = r'\lazada.xlsx';
un_process = r'\Unprocess';
data_link = r'\Data_link\data_link_all.json';
desk_top = 7;
staut_working = "test";

In [3]:
def is_thai(text):
    thai_unicode_range = (0x0E00, 0x0E7F)
    return all(ord(char) in range(thai_unicode_range[0], thai_unicode_range[1] + 1) for char in text)


In [13]:
def Check_header(path_file):
    try:
        check_data=[]
        df = pd.read_excel(path_file)
        print(df.columns[0])
        # is_subset = all(item in df.columns for item in header);
        # print("Check_data : พบข้อมูลทั้งหมดใน Excel ",is_subset);
        for i in range(len(header_2)):
            for excel in range(len(df.columns)):
                if(status_run_program):# หยุดทำงาน
                    return
                if(header_2[i]==df.columns[excel]):
                    check_data.append(df.columns[excel]);
        print("Check_data : ข้อมูลที่มีทั้งหมด ");
        return check_data; 
    except Exception as e:
        print("Check_data : ข้อมูล Excel ",e); 


In [19]:
def convert_to_integer(s):
    if 'k' or 'k+' in s:
        sum2 = (s.replace(' ชิ้น',''));
        sum3 = (sum2.replace('k+',''));
        sum3 = (sum3.replace('K',''));
        return float(sum3)*1000;
    elif '9,999+' in s:
        sum2 = (s.replace(' ชิ้น',''));
        sum3 = (sum2.replace('+',''));
        return float(sum3.replace(',',''))+1;
    else:
        return float(s.replace(' ชิ้น',''))
            
      
def get_datetime():
  now = datetime.now()
  date_str = now.strftime("%Y-%m-%d")
  time_str = now.strftime("%H:%M:%S")
  return {
    "date": date_str,
    "time": time_str,
}
def postAPI_DB(data,id_shop,link,date,time,website,group):
    try:
        response = requests.post(
            f'{uri_API}addb?idshop={id_shop}&&link={link}&&date={date}&&time={time}&&website={website}&&group={group}',
            headers={
                "Content-type":"application/x-www-form-urlencoded"
            },
            data={
                "data":data
            }
        )
        return response
    except:
        return {"status":404,"message":"POST API ERROR."}
# process excel file to Json(API)
def data_process(path_file,i1,i2,i3,group,link):
    """
    Process : เพื่อจัดตำแหน่งข้อมูลให้สามารถป้อนเข้า Data Base ได้
    Args:
        path_file: ไฟล์ข้อมูล excel ที่โหลดมาด้วยโปเกบอล อ่านเพื่อแปลงข้อมูลเป็น json
        i1: รอบการทำงานใหญ่ที่สุด ก็คือ หมวดหมู่
        i2: รอบการทำงานกลาง ก็คือ หมวดหมู่ สับย่อยลงมา
        i3: รอบการทำงานเล็กสุด ก็คือ หน้าแต่ละหน้าของสับย่อย ที่เราให้ไปอ่านแล้วกดโหลด
        group: _description_
    """
    try:
        header = Check_header(path_file)
        read_excel = pd.read_excel(path_file);
        num_rows, num_columns = read_excel.shape
        success_data_text = ""
        for i in range(num_rows):
            dt = get_datetime()
            Date = dt['date']
            Time = dt['time']
            data_process = {
                "product":[],
                "price_product_2":[],
                "price_product_1":[],
                "image_product_1":[],
                "discount":[],
                "image_product_2":[],
                "data_product":[],
                "price_before":[],
                "Emoji":[],
                "sold":[],
                "place":[],
                "Recommended_shops":[],
                "count_review":[],
                "maket":[],
                'gruop':[],
                'date':[],
                'time':[],
                'link':[]
            }
            data = "Product_"+str(i+1);
            for j in range(len(header)):
                data_input = str(read_excel[header[j]][i]);
                data_process[header_Values[header[j]]]=data_input;
            # ****************************************************************
            Product = {}
            Product[data]= data_process
            Product[data]["maket"]="lazada"
            Product[data]["group"]=group
            Product[data]['date'] = Date
            Product[data]['time'] = Time
            Product[data]['link'] = link
            id_shop = f'shop{i1}_{i2}_{i3}'

            # ****************************************************************
            product = Product[data]["product"]
            image_product_1 = Product[data]["image_product_1"]
            image_product_2 = Product[data]["image_product_2"]
            discount = Product[data]["discount"]
            data_product = Product[data]["data_product"]
            price_product = float(Product[data]["price_product"].replace("฿","").replace(",",""))
            price_product = (price_product<=0)and "0" or price_product
            if(len(Product[data]["sold"])>0):
                sold = (Product[data]["sold"].split(" ")[0]=="nan")and "0" or Product[data]["sold"].split(" ")[0];
            else:
                sold = "0"
            sold = convert_to_integer(sold);
            if is_thai(Product[data]["place"]):
                address = (Product[data]["place"]=='nan')and "" or Product[data]["place"]
            else:
                address = (Product[data]["place"]=='nan')and "" or ad[Product[data]["place"]]
            if 'จังหวัด' in address:
                address.replace("จังหวัด","")
            count_review = (Product[data]["count_review"]=="nan")and "0" or Product[data]["count_review"]
            maket = Product[data]["maket"]
            # ****************************************************************
            success_data_text += f'APRODUCT:::maket:::{maket},'
            success_data_text += f'group:::{group},'
            success_data_text += f'product:::{product},'
            success_data_text += f'price_product_2:::{""},' 
            success_data_text += f'price_product_1:::{price_product},'
            success_data_text += f'image_product_1:::{image_product_1},'
            success_data_text += f'discount:::{discount},'
            success_data_text += f'image_product_2:::{image_product_2},'
            success_data_text += f'data_product:::{data_product},'
            success_data_text += f'price_before:::{""},'
            success_data_text += f'Emoji:::{""},'
            success_data_text += f'sold:::{sold},'
            success_data_text += f'place:::{address},'
            success_data_text += f'Recommended_shops:::{""},'
            success_data_text += f'count_review:::{count_review},'
            success_data_text += f'date:::{Product[data]["date"]},'
            success_data_text += f'time:::{Product[data]["time"]},'
            success_data_text += f'link:::{Product[data]["link"]}'
            # ถ้าข้อมูลครบ 60 ค่อยบันทึก .json และส่ง API
        status_api = "";
        error_api = False;
        if(i==num_rows-1):
            for i in range(0,not error_api,3):
                status_api = postAPI_DB(success_data_text,id_shop,link,Date,Time,'Lazada',group);
                print(status_api.status_code);
                if(status_api.status_code==200):
                    error_api = True;
                    break
                else:
                    error_api = False;
            if(not error_api):
                   print("Hello")
            return error_api;
        # if(i==num_rows-1):
        #         print("Api : ส่งสำเร็จ");
        #         print(postAPI_DB(success_data_text,id_shop,link,Date,Time,'Lazada',group));
    except Exception as e:
        print(e);
data_process(path_file+data_lazada+r'/data_1_1_1.xlsx',1,2,3,"บ้านและไลฟ์สไตล์","Link_test");

_95X4G href
Check_data : ข้อมูลที่มีทั้งหมด 
https://www.lazada.co.th/products/oppq-a97-5g-67-hd2-smartphone-4g5g-16gb512gb-android-130-mobilephone-a97-5g-i5161520266.html
https://img.lazcdn.com/g/p/802bab51137346b707ee0fdb0ddf3192.jpg_80x80q80.jpg_.webp
โทรศัพท์มือถือ OPPQ A97 สมาร์ทโฟน 5G เครื่องใหม่ 6.7นิ้ว HD+รองรับ2ซิม Smartphone 4G/5G โทรศัพท์สมา แรม16GB+รอม512GB โทรศัพท์ถูกๆ Android 13.0 มือถือ Mobilephone มือถือราคาถูกๆ โทรศัพท์สำหรับเล่นเกม โทรสับราคาถูก A97 5G เเถมฟรีเคสใส+ฟิล์ม ในสต็อก
฿799.00
2.5K sold
(301)
Samut Sakhon
https://www.lazada.co.th/products/samsung-j2prime-5-15-8-6-2600-i4937026096.html
https://img.lazcdn.com/g/p/9c80e558aa9c30e3cd430d50fc49ad2f.jpg_80x80q80.jpg_.webp
Samsung​ J2prime​ หน้าจอ 5 นิ้ว แรม 1.5 รอม 8 แอนดรอย 6 แบตเตอรี่ 2600 แอมป์ มือถือ​พร้อมใช้งาน​ สภาพดี(ฟรีชุด​ชาร์จ)​
฿528.00
22 sold
(10)
Maha Sarakham
https://www.lazada.co.th/products/true-super-hero-4g-i3868143567.html
https://img.lazcdn.com/g/ff/kf/S586b49f88cdd4504b5e7fb26c5d21a55m.jpg_80x

In [9]:
print()

NameError: name 'success_data_text' is not defined

In [2]:
def get_datetime():
  now = datetime.now()
  date_str = now.strftime("%Y-%m-%d")
  time_str = now.strftime("%H:%M:%S")
  return {
    "date": date_str,
    "time": time_str,
}
print(get_datetime())

{'date': '2024-02-09', 'time': '21:20:34'}
